## Seven Days of Code - Day 3

### Data Cleaning

Day three of Seven Days of Code project
<p>More about the project <a href="https://7daysofcode.io/matricula/data-science">
here</a>.</p>

<p>Reference <a href="https://www.tableau.com/learn/articles/what-is-data-cleaning">
here</a> and <a href="http://www.kasimte.com/2020/02/09/linear-regression-from-time-series-data-using-scikit-learn.html"> here</a></p>


<p>Data Source <a href="https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps?utm_source=ActiveCampaign&utm_medium=email&utm_content=%237DaysOfCode+-+Ci%C3%AAncia+de+Dados+1%2F7%3A+Data+Cleaning+and+Preparation&utm_campaign=%5BAlura+%237Days+Of+Code%5D%28Java%29+Dia+1%2F7%3A+Consumir+uma+API+de+filmes" >here</a> </p>

### Goal: predict Brazilian Senators expenditure for certain days

In [1]:
#Import libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import datetime

import pandas as pd
import numpy as np

In [2]:
#Import the data

#ds == date of the reimbursement
#Y == sum of the reimbursements made in a certain date

data = pd.read_csv('dataset_ceaps_forecasting.csv')

In [3]:
data.head()

,ds,y
0,2018-01-01,13062.46
1,2018-01-02,61972.66
2,2018-01-03,54424.44
3,2018-01-04,21775.57
4,2018-01-05,41703.32


In [4]:
data.tail()

,ds,y
1456,2021-12-27,28552.41
1457,2021-12-28,154547.27
1458,2021-12-29,22495.76
1459,2021-12-30,118345.56
1460,2021-12-31,27219.41


#### Average of the last three months of data


In [5]:
#Take the year and the month from ds column
data['year'] = pd.DatetimeIndex(data['ds']).year
data['month'] = pd.DatetimeIndex(data['ds']).month

In [6]:
#Filtered dataframe
data_2021 = data.query('year == 2021')

In [7]:
data_2021

,ds,y,year,month
1096,2021-01-01,20971.94,2021,1
1097,2021-01-02,5863.44,2021,1
1098,2021-01-03,15293.99,2021,1
1099,2021-01-04,61860.13,2021,1
1100,2021-01-05,86977.14,2021,1
...,...,...,...,...
1456,2021-12-27,28552.41,2021,12
1457,2021-12-28,154547.27,2021,12
1458,2021-12-29,22495.76,2021,12
1459,2021-12-30,118345.56,2021,12


In [8]:
#Filter last three months
data_threemonths = data_2021.query('month in (12,11,10)')

In [9]:
data_threemonths

,ds,y,year,month
1369,2021-10-01,233702.29,2021,10
1370,2021-10-02,12978.01,2021,10
1371,2021-10-03,24074.49,2021,10
1372,2021-10-04,152114.12,2021,10
1373,2021-10-05,109587.07,2021,10
...,...,...,...,...
1456,2021-12-27,28552.41,2021,12
1457,2021-12-28,154547.27,2021,12
1458,2021-12-29,22495.76,2021,12
1459,2021-12-30,118345.56,2021,12


In [10]:
#Avg "y" for the last three months
print('The avg of reimbursements for the last three months is R$', round(data['y'].mean(),2))

The avg of reimbursements for the last three months is R$ 63876.04


### Linear Regression on Time Series

In [11]:
#Make sure "ds" is datetime
data['ds'] = pd.to_datetime(data['ds'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1461 non-null   datetime64[ns]
 1   y       1461 non-null   float64       
 2   year    1461 non-null   int64         
 3   month   1461 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 45.8 KB


In [12]:
#Set 'ds' as index
data.set_index('ds',inplace = True)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1461 entries, 2018-01-01 to 2021-12-31
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   y       1461 non-null   float64
 1   year    1461 non-null   int64  
 2   month   1461 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 45.7 KB


In [14]:
data.head()

,y,year,month
ds,,,
2018-01-01,13062.46,2018,1
2018-01-02,61972.66,2018,1
2018-01-03,54424.44,2018,1
2018-01-04,21775.57,2018,1
2018-01-05,41703.32,2018,1


In [15]:
#Convert the date index into an integer from the minimum start date
data['days_from_start'] = (data.index - data.index[0]).days;data

,y,year,month,days_from_start
ds,,,,
2018-01-01,13062.46,2018,1,0
2018-01-02,61972.66,2018,1,1
2018-01-03,54424.44,2018,1,2
2018-01-04,21775.57,2018,1,3
2018-01-05,41703.32,2018,1,4
...,...,...,...,...
2021-12-27,28552.41,2021,12,1456
2021-12-28,154547.27,2021,12,1457
2021-12-29,22495.76,2021,12,1458


In [16]:
#Simple Linear Regression
x = data['days_from_start'].values.reshape(-1,1)
y = data['y'].values

In [21]:
model = linear_model.LinearRegression().fit(x,y)
linear_model.LinearRegression(copy_X = True,n_jobs=1,normalize=False)
LinearRegression(copy_X=True,fit_intercept=True,n_jobs=1,normalize=False)
model.predict([[2],[12],[30]]).round(2)

array([65108.39, 65091.46, 65060.99])